In [21]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [22]:
reviews=pd.read_csv(r'C:\Users\work\Documents\womens-ecommerce-clothing-reviews\Womens Clothing E-Commerce Reviews.csv')
reviews.drop(['Unnamed: 0', 'Clothing ID','Age','Title','Recommended IND','Positive Feedback Count','Division Name','Department Name','Class Name'], axis=1, inplace=True)
reviews=reviews.dropna(how='any',axis=0)
reviews.head()

,Review Text,Rating
0,Absolutely wonderful - silky and sexy and comf...,4
1,Love this dress! it's sooo pretty. i happene...,5
2,I had such high hopes for this dress and reall...,3
3,"I love, love, love this jumpsuit. it's fun, fl...",5
4,This shirt is very flattering to all due to th...,5


In [23]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\work\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\work\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [24]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['Review Text'] = df['Review Text'].apply(lower_case)
        df['Review Text'] = df['Review Text'].apply(remove_stop_words)
        df['Review Text'] = df['Review Text'].apply(removing_numbers)
        df['Review Text'] = df['Review Text'].apply(removing_punctuations)
        df['Review Text'] = df['Review Text'].apply(removing_urls)
        df['Review Text'] = df['Review Text'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise


In [25]:
reviews = normalize_text(reviews)
reviews.head()

,Review Text,Rating
0,absolutely wonderful silky sexy comfortable,4
1,love dress sooo pretty happened find store i m...,5
2,high hope dress really wanted work me initiall...,3
3,love love love jumpsuit fun flirty fabulous ev...,5
4,shirt flattering due adjustable front tie perf...,5


In [26]:
reviews['Rating'].value_counts()

Rating
5    12540
4     4908
3     2823
2     1549
1      821
Name: count, dtype: int64

In [28]:
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(reviews['Review Text'])
y = reviews['Rating']

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow')
dagshub.init(repo_owner='kunalkaushik1982', repo_name='womens-ecommerce-clothing-reviews', mlflow=True)

mlflow.set_experiment("Logistic Regression Baseline")

Accessing as kunalkaushik1982

Initialized MLflow to track repo "kunalkaushik1982/womens-ecommerce-clothing-reviews"

Repository kunalkaushik1982/womens-ecommerce-clothing-reviews initialized!

2024/12/28 17:45:55 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/2ba21b33b41f47518d6ccc7d76408bef', creation_time=1735388156095, experiment_id='1', last_update_time=1735388156095, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [34]:
with mlflow.start_run():
    
    # Log preprocessing parameters
    mlflow.log_param("vectorizer", "Bag of Words")
    mlflow.log_param("num_features", 2000)
    mlflow.log_param("test_size", 0.2)
    
    # Model building and training
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    
    # Log model parameters
    mlflow.log_param("model", "Logistic Regression")
    
    # Model evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred,average='weighted')
    recall = recall_score(y_test, y_pred,average='weighted')
    f1 = f1_score(y_test, y_pred,average='weighted')
    
    # Log evaluation metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    
    # Log model
    mlflow.sklearn.log_model(model, "model")

    # Save and log the notebook
    import os
    notebook_path = "exp2_baseline_model.ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)
    
    # Print the results for verification
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

2024/12/28 17:51:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Accuracy: 0.637889158754692
Precision: 0.602859491839736
Recall: 0.637889158754692
F1 Score: 0.6134755115629251
🏃 View run enthused-mole-90 at: https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow/#/experiments/1/runs/490f690896dd4654959c0cf1896ed553
🧪 View experiment at: https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow/#/experiments/1
